In [4]:
# setup
import os, pandas as pd 
import matplotlib.pyplot as plt
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

plt.rcParams["figure.dpi"] = 140
plt.rcParams["axes.grid"] = True

load_dotenv()
engine = create_engine(os.getenv("DATABASE_URL"))

def q(sql, params=None):
    with engine.begin() as con:
        return pd.read_sql(text(sql), con, params=params)

In [ ]:
# Analyse average track features by genre
sql = """
SELECT
    st.playlist_genre,
    COUNT(*) AS n,
    AVG(st.danceability)::float AS avg_danceability,
    AVG(st.energy)::float AS avg_energy,
    AVG(st.valence)::float AS avg_valence,
    AVG(st.tempo)::float AS avg_tempo,
    AVG(st.loudness)::float AS avg_loudness_db
FROM staging_tracks st
GROUP BY st.playlist_genre
ORDER BY n DESC;
"""

df_genre = q(sql)
df_genre.head()

,playlist_genre,n,avg_danceability,avg_energy,avg_valence,avg_tempo,avg_loudness_db
0,rap,10802,0.715991,0.649832,0.505182,120.578818,-7.089071
1,pop,10264,0.637698,0.701031,0.502176,120.962740,-6.377052
2,edm,9754,0.657639,0.809604,0.397491,126.328088,-5.487192
3,r&b,9008,0.667475,0.588932,0.537936,113.916866,-8.033462
4,rock,8610,0.518519,0.733067,0.532560,125.039452,-7.559818
